First, we install the lyricsgenius API:

In [1]:
!pip install lyricsgenius

We import the libraries and set a path for our input file of artists:

In [1]:
from lyricsgenius import Genius
import json
import csv
import multiprocessing
import queue
import logging

# OS agnostic
import os 
CSV_PATH = os.path.join(os.path.curdir, 'artists', '10000-MTV-Music-Artists-page-%s.csv')

Now we import the artist dataset into an array. This consists of the top 10,000 artists listed on MTV:

We set up a lyricsgenius token, and use the API to pull the lyrics data for each artist in the dataset:

In [ ]:
# Genius setup
token = "EBufquOcw_ts4Y4V7yiddUNyUakTdqCpnMZhiI3XtAScWOntEom8Hj4T87gAV_cA"
genius = Genius(token, retries=2)

genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True
genius.excluded_terms = ["(Remix)", "(Live)"]

# Multiprocessing cores
process_number = int(multiprocessing.cpu_count()) * 2
multiprocessing.log_to_stderr().setLevel(logging.DEBUG)

# Data management
final_ = multiprocessing.Manager().list()

# artist_queue = queue.Queue()
# final_ = []
checked_artists = set()

# Pull out artists
def get_artists(queue):
    for x in range(1,5):
        path = CSV_PATH % str(x)
        with open(path, encoding="UTF-8") as csvfile:
            TopArtists = csv.reader(csvfile)
            
            # Skip header
            next(TopArtists)
            for row in TopArtists:
                artist = row[0]
                # Check if we should skip this artists since we already found the data
                if artist not in checked_artists:
                    queue.put(artist)
                      


# File management
def write_to_csv(data, columns = ["artist", "song", "data"], file_name="song_data.csv"):
    """
    data: list of dictionaries {artist, song, data}
    """
    
    csv_path = os.path.join(os.path.curdir, 'data', file_name)
    with open(csv_path, 'w') as csv_file: 
        # creating a csv dict writer object 
        writer = csv.DictWriter(csv_file, fieldnames = columns) 
        
        # writing headers (field names) 
        writer.writeheader() 
        
        # writing data rows 
        writer.writerows(data) 
        

def read_csv(file_name="song_lyrics.csv"):
    global final_, checked_artists     
    
    csv_path = os.path.join(os.path.curdir, 'data', file_name)
    artist_set = set()
    
    # opening the CSV file
    try:
        with open(csv_path, mode ='r') as file:   

            # reading the CSV file
            data = csv.DictReader(file)

            for entry in data:
                artist_set.add(entry["artist"])
                final_.append(entry)
                
    except FileNotFoundError:
        pass
    
# Processing
def clean_data(data):
    cleaned_data = data.replace("\n", " ").replace(",", " ")
    return cleaned_data

def process_artist(artist):
    artist_dict = artist.to_dict()
    return ""

def process_song(song):
    lyrics = clean_data(song.lyrics)
    return lyrics
                           
def build_entry(artist, song, data, columns = ["artist", "song", "data"]):
    entry = [{"artist": artist, "song": song, "data": data}]
    

# Run genius search
def search_genius(args):
    artist_queue, num = args
    print("[{num}] Starting\n".format(num=num), end='')
    try:
        while True:
            artist = artist_queue.get()
            if artist is None:
                print("[{num}] Done\n".format(num=num), end='')
                return
            print("[{num}] Searching {artist}\n".format(num=num, artist=artist.strip()), end='')
            
            # Pull data for artist from genius
            genius_artist = genius.search_artist(artist, per_page=50, get_full_info=False)
            print("[{num}] Done".format(num=num), end='')              
            if genius_artist == None:
                print("[{num}] {artist} not found\n".format(num=num, artist=artist.strip()), end='')
                continue
                           
            artist_data =  process_artist(genius_artist)
                           
            print("[{num}] {artist} number of songs: {song_num}\n".format(num=num, artist=artist.strip(), song_num=len(genius_artist.songs)), end='')
            
            for song in genius_artist.songs:
                song_data = process_song(song)
                
                # Add to final list
                final_.append(build_entry(artist, song, song_data))
    
    except Exception as e:
        print("[{num}] Something went wrong: {error}\n".format(num=num, error= e), end='')
    
    
def run(multi_core=False):                  
    
    # Load in any previous data
    print("Reading previous")
    read_csv()
    
    pool = None
    try:  
        if multi_core:
            print("Multiprocessing with {process_number} processes".format(process_number=process_number))
            
            artist_queue = multiprocessing.Manager().Queue()
            get_artists(artist_queue)
            
            for x in range(process_number):
                artist_queue.put(None)
            
            print(artist_queue.qsize())
            # creating processes
            pool = multiprocessing.Pool(process_number)
            args = [(artist_queue, x) for x in range(process_number)]
            pool.map(search_genius, args)
            pool.join()
            
        else:
            print("Running single core")
            artist_queue = queue.Queue()
            get_artists(artist_queue)
            rtist_queue.put(None)
            print(artist_queue.qsize())
            search_genius((1, artist_queue))

    
    except KeyboardInterrupt:
        if pool:
            pool.close()
            pool.terminate()
            pool.join()
        print("KeyboardInterrupt: Writing results")
    
    finally:
        write_to_csv(list(final_))                       


run(multi_core=True) 




[INFO/SyncManager-1] child process calling self.run()
[INFO/SyncManager-1] created temp directory /var/folders/jc/85vy06cn6b13gcm4rcvz9_fw0000gq/T/pymp-2c1gh2ix
[INFO/SyncManager-1] manager serving at '/var/folders/jc/85vy06cn6b13gcm4rcvz9_fw0000gq/T/pymp-2c1gh2ix/listener-w0xzvj21'
[DEBUG/MainProcess] requesting creation of a shared 'list' object
[DEBUG/SyncManager-1] 'list' callable returned object with id '109298fc8'
[DEBUG/MainProcess] INCREF '109298fc8'


Reading previous
Multiprocessing with 32 processes


[DEBUG/SyncManager-2] INCREF '109298fc8'
[INFO/SyncManager-2] child process calling self.run()
[INFO/SyncManager-2] created temp directory /var/folders/jc/85vy06cn6b13gcm4rcvz9_fw0000gq/T/pymp-021lnn9z
[DEBUG/MainProcess] requesting creation of a shared 'Queue' object
[INFO/SyncManager-2] manager serving at '/var/folders/jc/85vy06cn6b13gcm4rcvz9_fw0000gq/T/pymp-021lnn9z/listener-aptz2jcy'
[DEBUG/SyncManager-2] 'Queue' callable returned object with id '1092c8828'
[DEBUG/MainProcess] INCREF '1092c8828'
[DEBUG/MainProcess] thread 'MainThread' does not own a connection
[DEBUG/MainProcess] making connection to manager
[DEBUG/SyncManager-2] starting server thread to service 'MainProcess'
[DEBUG/MainProcess] created semlock with handle 64
[DEBUG/MainProcess] created semlock with handle 65
[DEBUG/MainProcess] created semlock with handle 68
[DEBUG/MainProcess] created semlock with handle 69


8376


[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] added worker
[DEBUG/ForkPoolWorker-3] INCREF '109298fc8'[DEBUG/MainProcess] added worker

[DEBUG/ForkPoolWorker-4] INCREF '109298fc8'[DEBUG/MainProcess] added worker
[DEBUG/ForkPoolWorker-3] INCREF '1092c8828'

[DEBUG/MainProcess] added worker
[INFO/ForkPoolWorker-3] child process calling self.run()[DEBUG/ForkPoolWorker-4] INCREF '1092c8828'[DEBUG/ForkPoolWorker-5] INCREF '109298fc8'[DEBUG/MainProcess] added worker



[DEBUG/MainProcess] added worker
[INFO/ForkPoolWorker-4] child process calling self.run()
[DEBUG/ForkPoolWorker-5] INCREF '1092c8828'[DEBUG/MainProcess] added worker
[DEBUG/ForkPoolWorker-6] INCREF '109298fc8'

[INFO/ForkPoolWorker-5] child process calling self.run()[DEBUG/MainProcess] added worker

[DEBUG/ForkPoolWorker-6] INCREF '1092c8828'[DEBUG/ForkPoolWorker-7] INCREF '109298fc8'[DEBUG/MainProcess] added worker

[INFO/ForkPoolWorker-6] child process calling self.run()
[DEBUG/MainProcess] added worker

[DEBUG/ForkPo

...


[DEBUG/MainProcess] added worker
[DEBUG/ForkPoolWorker-36] INCREF '1092c8828'
[DEBUG/ForkPoolWorker-37] INCREF '109298fc8'[DEBUG/SyncManager-2] starting server thread to service 'ForkPoolWorker-34'

[30] Searching Luke Bryan


[INFO/ForkPoolWorker-36] child process calling self.run()


[DEBUG/MainProcess] added worker


[31] Searching Blake Shelton


[DEBUG/SyncManager-2] got EOF -- exiting thread serving 'ForkPoolWorker-3'[DEBUG/ForkPoolWorker-37] INCREF '1092c8828'
[DEBUG/ForkPoolWorker-38] INCREF '109298fc8'[DEBUG/MainProcess] added worker
[DEBUG/SyncManager-2] got EOF -- exiting thread serving 'ForkPoolWorker-4'


[INFO/ForkPoolWorker-37] child process calling self.run()[DEBUG/SyncManager-2] got EOF -- exiting thread serving 'ForkPoolWorker-5'[DEBUG/ForkPoolWorker-38] INCREF '1092c8828'

[DEBUG/SyncManager-2] got EOF -- exiting thread serving 'ForkPoolWorker-8'
[DEBUG/MainProcess] added worker
[DEBUG/ForkPoolWorker-39] INCREF '109298fc8'
[INFO/ForkPoolWorker-38] child process calling self.run()
[DEBUG/SyncManager-2] got EOF -- exiting thread serving 'ForkPoolWorker-6'
[DEBUG/MainProcess] added worker

[DEBUG/ForkPoolWorker-39] INCREF '1092c8828'[DEBUG/SyncManager-2] got EOF -- exiting thread serving 'ForkPoolWorker-7'
[DEBUG/ForkPoolWorker-40] INCREF '109298fc8'[INFO/ForkPoolWorker-39] child process calling self.run()


[DEBUG/